In [1]:
# Import libraries
from gurobipy import*

import math
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

In [2]:
# File path 
FILE = "./DaSilvaUrrutia/n200w100.001.txt"

In [3]:
# Default params
SUPPORTED_FORMAT = ['NUM', 'X', 'Y', 'DEMAND', 'READYTIME', 'DUEDATE', 'SERVICE']
MINUTES = 60
HOURS = 3600
OFFSET_TIMES = 8*HOURS

COLUMNS_OPS = {'NUM': lambda x: float(x),
               'X': lambda x: float(x),
               'Y': lambda x: float(x),
               'DEMAND': lambda x: 1,
               'READYTIME': lambda x: float(x)+OFFSET_TIMES,
               'DUEDATE': lambda x: float(x)+OFFSET_TIMES,
               'SERVICE': lambda x: 30*MINUTES
              }

AGENTS = 20

TIME_PER_DISTANCE = 1

WORKING_TIME_RANGE = (OFFSET_TIMES, OFFSET_TIMES + 8*HOURS)

LUNCH_BREAK_RANGE = (12*HOURS, 13.5*HOURS)
LUNCH_BREAK_TIME = 30*MINUTES

OFFICE_NUM = 0
OFFICE_X = .0
OFFICE_Y = .0
OFFICE_READYTIME = WORKING_TIME_RANGE[0]
OFFICE_DUEDATE = WORKING_TIME_RANGE[1]
OFFICE_SERVICE = 1*HOURS

In [4]:
def read_input_tsptw(filename):
    """This function is used to convert input file to usable data"""
    nb_nodes = 0
    
    data_dict = dict()
    
    nodes_x = list()
    nodes_y = list()
       
    # Open file and read lines 
    with open(filename, "r") as file:
        # Initialize columns in empty dict
        columns = file.readline().replace("#","").split()
        if columns != SUPPORTED_FORMAT:
            print("ERROR! Format not supported.")
            return 
        
        # Add office to data
        data_dict.update({OFFICE_NUM: {'X': OFFICE_X, 
                                       'Y': OFFICE_Y, 
                                       'READYTIME': OFFICE_READYTIME,
                                       'DUEDATE': OFFICE_DUEDATE,
                                       'SERVICE': OFFICE_SERVICE,}})
        # Add office to nodes
        nodes_x.append(OFFICE_X)
        nodes_y.append(OFFICE_Y)
            
        # For each data line
        for line in file.readlines():
            node_dict = {k: COLUMNS_OPS[k](val) for k, val in zip(columns, line.split())}
                
            # Get id
            node_id = node_dict.pop('NUM')
            # Insert new node in data dict
            data_dict.update({int(node_id): node_dict})            
            # Get nodes positions
            nodes_x.append(float(line.split()[columns.index('X')]))
            nodes_y.append(float(line.split()[columns.index('Y')]))

    # Get distance matrix
    distance_matrix = compute_distance_matrix(nodes_x, nodes_y)
    
    return (data_dict, distance_matrix, dict(enumerate(zip(nodes_x, nodes_y))))


def compute_distance_matrix(nodes_x, nodes_y):
    """This function is used to compute the distance matrix"""
    nb_customers = len(nodes_x)
    distance_matrix = [[None for i in range(nb_customers)] for j in range(nb_customers)]
    for i in range(nb_customers):
        distance_matrix[i][i] = 0
        for j in range(nb_customers):
            dist = compute_dist(nodes_x[i], nodes_x[j], nodes_y[i], nodes_y[j])
            distance_matrix[i][j] = dist
            distance_matrix[j][i] = dist
    return distance_matrix


def compute_dist(xi, xj, yi, yj):
    """This function is used to compute euclidean distance"""
    exact_dist = math.sqrt(math.pow(xi - xj, 2) + math.pow(yi - yj, 2))
    return int(math.floor(exact_dist + 0.5)) * TIME_PER_DISTANCE

In [5]:
# Getting parameters
data_dict, distance_matrix, positions = read_input_tsptw(FILE)

In [6]:
# DEBUG RESTRICTIONS

AGENTS=10
data_dict = {k: v for k,v in data_dict.items() if k < 20}
distance_matrix = distance_matrix[:20][:20]


### Define function to plot the graph

In [7]:
def plot_model(model, positions):
    """This function is used to plot graph"""
    # Create figure
    plt.figure(figsize=(16,9))
    G = nx.Graph()
    # Add nodes to graph
    G.add_nodes_from(positions.keys())
    
    # Set labels dict
    node_labels = dict()
    # Set nodes positions
    for n, p in positions.items():
        G.nodes[n]['pos'] = p
        node_labels[n] = n

    # Define edges colors
    edge_colors = [plt.cm.tab20.colors[i] for i in model.keys()]
    # Add edges
    for t, l in model.items():
        for i in range(1,len(l['trip'])):
            G.add_edge(l['trip'][i-1], l['trip'][i], 
                       color=edge_colors[t], 
                       alpha=0.5, 
                       weight=4)
    # Set edges parameters 
    edges = G.edges()
    colors = [G[u][v]['color'] for u,v in edges]
    weights = [G[u][v]['weight'] for u,v in edges]
    alphas = [G[u][v]['alpha'] for u,v in edges]

    # Draw Nodes
    nx.draw_networkx_nodes(G, positions,
                           node_size=200,
                           node_color='c')
    # Draw Labels
    nx.draw_networkx_labels(G, positions, 
                            node_labels, 
                            font_size=10, 
                            font_color='k')
    # Draw Edges 
    nx.draw_networkx_edges(G, positions, 
                           alpha=0.5, 
                           width=weights, 
                           edge_color=colors)

    # Show graph
    plt.show()

In [8]:
# Create model
mod = Model("TSPTW")     

Using license file c:\gurobi903\gurobi.lic
Academic license - for non-commercial use only


In [9]:
# Add variables
#  Client served
x = mod.addVars({(i,j,a): 0 for i in range(len(data_dict)) 
                            for j in range(len(data_dict)) 
                            for a in range(AGENTS)}, 
                name="x", 
                vtype=GRB.INTEGER)

#  Wait time
y = mod.addVars({(i,a): 0 for i in range(len(data_dict)) 
                          for a in range(AGENTS)}, 
                name="y", 
                vtype=GRB.INTEGER)

# #  Minutes in office
# z = mod.addVars({(i,j,a): 0 for i in range(len(data_dict)) 
#                             for j in range(len(data_dict)) 
#                             for a in range(AGENTS)}, 
#                 name="z", 
#                 vtype=GRB.INTEGER)

# Service time of client i
s =  mod.addVars({(i,a): 0 for i in range(len(data_dict)) 
                           for a in range(AGENTS)}, 
                 name="s", 
                 vtype=GRB.INTEGER)

In [10]:
# Add constraints
_= mod.addConstrs((quicksum(x[i,j,a] for a in range(AGENTS)
                                     for j in range(len(data_dict))) == 1
                  for i in range(len(data_dict))),
                  name="ServeAll")

_= mod.addConstrs((quicksum(x[i,j,a] for i in range(len(data_dict)) 
                                     for a in range(AGENTS)) == 1
                  for j in range(len(data_dict))),
                  name="ServeOnce")

_= mod.addConstrs((quicksum(x[i,j,a]+x[j,i,a] for a in range(AGENTS)) <= 1
                  for i in range(len(data_dict))
                  for j in range(len(data_dict))),
                  name="NoLoops")

# _= mod.addConstrs((quicksum(x[i,0,a] * z[i,0,a] 
#                                     for i in range(len(data_dict))) == data_dict[0]['SERVICE']
#                   for a in range(AGENTS)),
#                   name="CompleteOfficeHours")

_= mod.addConstrs((quicksum(x[i,h,a]
                           for i in range(len(data_dict))) -
                   quicksum(x[h,j,a]
                           for j in range(len(data_dict))) == 0
                   for h in range(len(data_dict))
                   for a in range(AGENTS)),
                  name="EachClientHasOneBefore")

_= mod.addConstrs((quicksum(x[i,i,a]
                           for i in range(len(data_dict))) == 0
                   for a in range(AGENTS)),
                  name="NotSame")

_= mod.addConstrs((quicksum(x[i,j,a] * distance_matrix[i][j]
                           for i in range(len(data_dict))
                           for j in range(len(data_dict))) +
#                    quicksum(x[i,j,a] * y[j,a]
#                            for i in range(len(data_dict))
#                            for j in range(len(data_dict))) +
                   quicksum(x[i,j,a] * data_dict[j]['SERVICE']
                           for i in range(len(data_dict))
                           for j in range(1,len(data_dict))) +
#                    quicksum(x[i,0,a] * z[i,0,a]
#                            for i in range(len(data_dict))) +
                   WORKING_TIME_RANGE[0] <= WORKING_TIME_RANGE[1]
                   for a in range(AGENTS)),
                  name="MaxHours")

# _= mod.addConstrs((quicksum(x[i,i,a]
#                            for i in range(len(data_dict))) == 0
#                    for a in range(AGENTS)),
#                   name="TripToAnother")

RANGE_MAX =max([data_dict[i]['DUEDATE'] + distance_matrix[i][j] - data_dict[j]['READYTIME']
                       for i in range(len(data_dict))
                       for j in range(len(data_dict))])

_= mod.addConstrs((s[i,a] + distance_matrix[i][j] - RANGE_MAX * (1 - x[i,j,a]) + y[j,a] <= s[j,a]
                   for i in range(len(data_dict))
                   for j in range(len(data_dict))
                   for a in range(AGENTS)),
                  name="RouteInTime")

_= mod.addConstrs((data_dict[i]['READYTIME'] <= s[i,a] + y[i,a]
                   for i in range(len(data_dict))
                   for a in range(AGENTS)),
                  name="ServeAfterTWStart")

_= mod.addConstrs((data_dict[i]['DUEDATE'] >= s[i,a]
                   for i in range(len(data_dict))
                   for a in range(AGENTS)),
                  name="ServeBeforeTWEnd")

_= mod.addConstrs((data_dict[j]['READYTIME'] - x[i,j,a] * s[j,a] == y[j,a]
                   for i in range(len(data_dict))
                   for j in range(len(data_dict))
                   for a in range(AGENTS)),
                  name="WaitForClient")

In [11]:
COST_WAIT = 2
# Add objective function
mod.setObjective(quicksum(x[i,j,a]*distance_matrix[i][j] # + y[j,a] * COST_WAIT
                                                         for i in range(len(data_dict))
                                                         for j in range(len(data_dict))
                                                         for a in range(AGENTS)), 
                 GRB.MINIMIZE)
    
# Optimize the model
mod.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 5060 rows, 4400 columns and 43790 nonzeros
Model fingerprint: 0xdec7a2b6
Model has 4000 quadratic constraints
Variable types: 0 continuous, 4400 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+04]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [4e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 6e+04]
  QRHS range       [3e+04, 4e+04]
Presolve removed 430 rows and 0 columns
Presolve time: 0.01s

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 12 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [12]:
mod.computeIIS()


Computing Irreducible Inconsistent Subsystem (IIS)...

      Constraints           Bounds       Runtime
     Min       Max       Min      Max
------------------------------------------------
        0     9060         0     4400         0s
        5        5       202      202         2s

IIS computed: 5 constraints, 202 bounds
IIS runtime: 2.35 seconds


In [13]:
for a in range(AGENTS):
    sss = list()
    for i in range(len(data_dict)):
        for j in range(len(data_dict)):
            if x[i,j,a].X:
                sss.append((i,j))
    print(f"Agent {a}: {sss}")

AttributeError: Unable to retrieve attribute 'X'